In [562]:
import sqlite3

# In this tutorial we will use the well-known Northwind sample database (included in MS Access and MS SQL Server).
# northwind.sql 쿼리 파일과 SQLite CLI를 이용해 DB 파일을 생성한다. ex) sqlite3 northwind.db < northwind.sql
tutorial_db = 'northwind.db'

In [563]:
def execute_example(query: str, N: int = None):
    with sqlite3.connect(tutorial_db) as conn:
        if N is not None:
            query = f"{query.strip().rstrip(';')} LIMIT {N};"

        cursor = conn.cursor()
        cursor.execute(query)
        column_names = [description[0] for description in cursor.description]
        rows = cursor.fetchall()

        print_query_result(column_names, rows)
        print()

def execute_rollback_example(insert_query: str, select_query: str):
    with sqlite3.connect(tutorial_db) as conn:
            cursor = conn.cursor()
            cursor.execute(insert_query)
            cursor.execute(select_query)

            column_names = [description[0] for description in cursor.description]
            rows = cursor.fetchall()
            
            print_query_result(column_names, rows)
            print()

            # 예제의 멱등성을 보장하기 위한 롤백
            conn.rollback()

def print_query_result(column_names: list, rows: list):
    print(" | ".join(column_names))
    print("-" * (len(column_names) * 15))
    
    for row in rows:
        print(" | ".join(str(cell) for cell in row))

## Demo Table 생성 예제

In [564]:
def create_demo_table(conn: sqlite3.Connection ,create_query: str, insert_query: str, data: list):
    cursor = conn.cursor()
    cursor.execute(create_query)
    cursor.executemany(insert_query, data)
    conn.commit()

# Create Demo Table 예제
create_query = """
    CREATE TABLE IF NOT EXISTS Customers (
        CustomerID INTEGER PRIMARY KEY,
        CustomerName TEXT NOT NULL,
        ContactName TEXT NOT NULL,
        Address TEXT NOT NULL,
        City TEXT NOT NULL,
        PostalCode TEXT NOT NULL,
        Country TEXT NOT NULL
    );
"""

insert_query = """
INSERT INTO Customers (CustomerID, CustomerName, ContactName, Address, City, PostalCode, Country)
VALUES (?, ?, ?, ?, ?, ?, ?);
"""

data = [
    (1, 'Alfreds Futterkiste', 'Maria Anders', 'Obere Str. 57', 'Berlin', '12209', 'Germany'),
    (2, 'Ana Trujillo Emparedados y helados', 'Ana Trujillo', 'Avda. de la Constitución 2222', 'México D.F.', '05021', 'Mexico'),
    (3, 'Antonio Moreno Taquería', 'Antonio Moreno', 'Mataderos 2312', 'México D.F.', '05023', 'Mexico'),
    (4, 'Around the Horn', 'Thomas Hardy', '120 Hanover Sq.', 'London', 'WA1 1DP', 'UK'),
    (5, 'Berglunds snabbköp', 'Christina Berglund', 'Berguvsvägen 8', 'Luleå', 'S-958 22', 'Sweden'),
    (6, 'Blauer See Delikatessen', 'Hanna Moos', 'Forsterstr. 57', 'Mannheim', '68306', 'Germany'),
    (7, 'Blondel père et fils', 'Frédérique Citeaux', '24, place Kléber', 'Strasbourg', '67000', 'France'),
    (8, 'Bólido Comidas preparadas', 'Martín Sommer', 'C/ Araquil, 67', 'Madrid', '28023', 'Spain'),
    (9, "Bon app'", 'Laurence Lebihans', '12, rue des Bouchers', 'Marseille', '13008', 'France'),
    (10, 'Bottom-Dollar Marketse', 'Elizabeth Lincoln', '23 Tsawassen Blvd.', 'Tsawassen', 'T2F 8M4', 'Canada'),
    # ...    
]

conn = sqlite3.connect(":memory:")
create_demo_table(conn, create_query, insert_query, data)

cursor = conn.cursor()
cursor.execute("SELECT * FROM Customers;")
for row in cursor.fetchall():
    print(row)

(1, 'Alfreds Futterkiste', 'Maria Anders', 'Obere Str. 57', 'Berlin', '12209', 'Germany')
(2, 'Ana Trujillo Emparedados y helados', 'Ana Trujillo', 'Avda. de la Constitución 2222', 'México D.F.', '05021', 'Mexico')
(3, 'Antonio Moreno Taquería', 'Antonio Moreno', 'Mataderos 2312', 'México D.F.', '05023', 'Mexico')
(4, 'Around the Horn', 'Thomas Hardy', '120 Hanover Sq.', 'London', 'WA1 1DP', 'UK')
(5, 'Berglunds snabbköp', 'Christina Berglund', 'Berguvsvägen 8', 'Luleå', 'S-958 22', 'Sweden')
(6, 'Blauer See Delikatessen', 'Hanna Moos', 'Forsterstr. 57', 'Mannheim', '68306', 'Germany')
(7, 'Blondel père et fils', 'Frédérique Citeaux', '24, place Kléber', 'Strasbourg', '67000', 'France')
(8, 'Bólido Comidas preparadas', 'Martín Sommer', 'C/ Araquil, 67', 'Madrid', '28023', 'Spain')
(9, "Bon app'", 'Laurence Lebihans', '12, rue des Bouchers', 'Marseille', '13008', 'France')
(10, 'Bottom-Dollar Marketse', 'Elizabeth Lincoln', '23 Tsawassen Blvd.', 'Tsawassen', 'T2F 8M4', 'Canada')


## SQL Examples

### SQL SELECT Statement

In [565]:
with sqlite3.connect(tutorial_db) as conn:
    cursor = conn.cursor()

    query = '''
        SELECT CustomerName, City FROM Customers;
    '''
    cursor.execute(query)
    rows = cursor.fetchall()
    for row in rows[:5]:
        print(row)
    conn.rollback()

    cursor.close()

('Alfreds Futterkiste', 'Berlin')
('Ana Trujillo Emparedados y helados', 'México D.F.')
('Antonio Moreno Taquería', 'México D.F.')
('Around the Horn', 'London')
('Berglunds snabbköp', 'Luleå')


### SQL SELECT DISTINCT Statement

In [566]:
sub_query = '''
    CREATE TEMP TABLE Demo AS
    SELECT City, Country
    FROM Customers
    ORDER BY CustomerID
    LIMIT 5;
    '''

def distinct_example(title: str, query: str):
    with sqlite3.connect(tutorial_db) as conn:
        print(title)
        cursor = conn.cursor()

        cursor.execute(sub_query)

        cursor.execute(query)
        column_names = [description[0] for description in cursor.description]
        rows = cursor.fetchall()
    
        print_query_result(column_names, rows)
        print()

# Distinct
title = "DISTNCT in 5 Records"
query = '''
            SELECT DISTINCT Country FROM Demo;
        '''
distinct_example(title, query)

# Without DISTINCT
title = "Without DISTINCT in 5 Records"
query = '''
            SELECT Country FROM Demo;
        '''
distinct_example(title, query)

# Count DISTINCT
title = "Count DISTINCT"
query = '''
            SELECT COUNT(DISTINCT Country) FROM Customers;
        '''
distinct_example(title, query)

# Count without DISTINCT
title = "Count DISTINCT"
query = '''
            SELECT COUNT(Country) FROM Customers;
        '''
distinct_example(title, query)

DISTNCT in 5 Records
Country
---------------
Germany
Mexico
UK
Sweden

Without DISTINCT in 5 Records
Country
---------------
Germany
Mexico
Mexico
UK
Sweden

Count DISTINCT
COUNT(DISTINCT Country)
---------------
21

Count DISTINCT
COUNT(Country)
---------------
91



### SQL WHERE Clause

In [567]:
query = '''
    SELECT * FROM Customers
    WHERE Country='Mexico';
'''
execute_example(query)

# Numeric Fields in The WHERE Clause, numeric fields should not be enclosed in quotes:
query = '''
    SELECT * FROM Customers
    WHERE CustomerID=1;
'''
execute_example(query)

# Operators in The WHERE Clause, 논리 연산자와 범위 및 패턴 연산자를 Where 절에 사용할 수 있다.
query = '''
    SELECT * FROM Customers
    WHERE CustomerID > 80;
'''
execute_example(query)

CustomerID | CustomerName | ContactName | Address | City | PostalCode | Country
---------------------------------------------------------------------------------------------------------
2 | Ana Trujillo Emparedados y helados | Ana Trujillo | Avda. de la Constitución 2222 | México D.F. | 5021 | Mexico
3 | Antonio Moreno Taquería | Antonio Moreno | Mataderos 2312 | México D.F. | 5023 | Mexico
13 | Centro comercial Moctezuma | Francisco Chang | Sierras de Granada 9993 | México D.F. | 5022 | Mexico
58 | Pericles Comidas clásicas | Guillermo Fernández | Calle Dr. Jorge Cash 321 | México D.F. | 5033 | Mexico
80 | Tortuga Restaurante | Miguel Angel Paolino | Avda. Azteca 123 | México D.F. | 5033 | Mexico

CustomerID | CustomerName | ContactName | Address | City | PostalCode | Country
---------------------------------------------------------------------------------------------------------
1 | Alfreds Futterkiste | Maria Anders | Obere Str. 57 | Berlin | 12209 | Germany

CustomerID | CustomerNa

### The SQL ORDER BY

In [568]:
print("ASC(Default) ORDER BY")
query = '''
    SELECT * FROM Products
    ORDER BY Price;
'''
execute_example(query, N=5)

print("DESC ORDER BY")
query = '''
    SELECT * FROM Products
    ORDER BY Price DESC;
'''
execute_example(query, N=5)

print("Alphabetically ORDER BY")
query = '''
    SELECT * FROM Products
    ORDER BY ProductName;
'''
execute_example(query, N=5)

print("Alphabetically DESC ORDER BY")
query = '''
    SELECT * FROM Products
    ORDER BY ProductName DESC;
'''
execute_example(query, N=5)

print("ORDER BY Several Columns")
query = '''
    SELECT * FROM Customers
    ORDER BY Country, CustomerName;
'''
execute_example(query, N=5)

print("Using Both ASC and DESC")
query = '''
    SELECT * FROM Customers
    ORDER BY Country ASC, CustomerName DESC;
'''
execute_example(query, N=5)

ASC(Default) ORDER BY
ProductID | ProductName | SupplierID | CategoryID | Unit | Price
------------------------------------------------------------------------------------------
33 | Geitost | 15 | 4 | 500 g | 2.5
24 | Guaraná Fantástica | 10 | 1 | 12 - 355 ml cans | 4.5
13 | Konbu | 6 | 8 | 2 kg box | 6
52 | Filo Mix | 24 | 5 | 16 - 2 kg boxes | 7
54 | Tourtière | 25 | 6 | 16 pies | 7.45

DESC ORDER BY
ProductID | ProductName | SupplierID | CategoryID | Unit | Price
------------------------------------------------------------------------------------------
38 | Côte de Blaye | 18 | 1 | 12 - 75 cl bottles | 263.5
29 | Thüringer Rostbratwurst | 12 | 6 | 50 bags x 30 sausgs. | 123.79
9 | Mishi Kobe Niku | 4 | 6 | 18 - 500 g pkgs. | 97
20 | Sir Rodney's Marmalade | 8 | 3 | 30 gift boxes | 81
18 | Carnarvon Tigers | 7 | 8 | 16 kg pkg. | 62.5

Alphabetically ORDER BY
ProductID | ProductName | SupplierID | CategoryID | Unit | Price
-------------------------------------------------------------

### The SQL AND Operator
1. The WHERE clause can contain one or many AND operators.
2. The AND operator is used to filter records based on more than one condition.
3. The AND operator displays a record if **all the conditions** are TRUE.

In [569]:
print("The SQL AND Operator")
# Select all customers from Spain that starts with the letter 'G':
query = '''
    SELECT *
    FROM Customers
    WHERE Country = 'Spain' AND CustomerName LIKE 'G%';
'''
execute_example(query)

print("All Conditions Must Be True")
# The following SQL statement selects all fields from Customers
# where Country is "Germany" AND City is "Berlin" AND PostalCode is higher than 12000:
query = '''
    SELECT * FROM Customers
    WHERE Country = 'Germany'
    AND City = 'Berlin'
    AND PostalCode > 12000;
'''
execute_example(query)

print("Combining AND and OR")
# Select all Spanish customers that starts with either "G" or "R":
query = '''
    SELECT * FROM Customers
    WHERE Country = 'Spain' AND (CustomerName LIKE 'G%' OR CustomerName LIKE 'R%');
'''
execute_example(query)

print("Combining AND and OR with Parentheses")
# Select all Spanish customers that starts with either "G" or "R":
query = '''
    SELECT * FROM Customers
    WHERE Country = 'Spain' AND (CustomerName LIKE 'G%' OR CustomerName LIKE 'R%');
'''
execute_example(query)

print("Combining AND and OR without Parentheses")
# Select all customers that either:
# are from Spain and starts with either "G", or
# starts with the letter "R":
query = '''
    SELECT * FROM Customers
    WHERE Country = 'Spain' AND CustomerName LIKE 'G%' OR CustomerName LIKE 'R%';
'''
execute_example(query)

The SQL AND Operator
CustomerID | CustomerName | ContactName | Address | City | PostalCode | Country
---------------------------------------------------------------------------------------------------------
29 | Galería del gastrónomo | Eduardo Saavedra | Rambla de Cataluña, 23 | Barcelona | 8022 | Spain
30 | Godos Cocina Típica | José Pedro Freyre | C/ Romero, 33 | Sevilla | 41101 | Spain

All Conditions Must Be True
CustomerID | CustomerName | ContactName | Address | City | PostalCode | Country
---------------------------------------------------------------------------------------------------------
1 | Alfreds Futterkiste | Maria Anders | Obere Str. 57 | Berlin | 12209 | Germany

Combining AND and OR
CustomerID | CustomerName | ContactName | Address | City | PostalCode | Country
---------------------------------------------------------------------------------------------------------
29 | Galería del gastrónomo | Eduardo Saavedra | Rambla de Cataluña, 23 | Barcelona | 8022 | Spain
30 

### The SQL OR Operator
1. The WHERE clause can contain one or many OR operators.
2. The OR operator is used to filter records based on more than one condition.
3. The OR operator displays a record if **any the conditions** are TRUE.

In [570]:
print("The SQL OR Operator")
# Select all customers from Germany or Spain:
query = '''
   SELECT *
    FROM Customers
    WHERE Country = 'Germany' OR Country = 'Spain';
'''
execute_example(query)

print("At Least One Condition Must Be True")
# selects all fields from Customers
# where either City is "Berlin", CustomerName starts with the letter "G" or Country is "Norway":
query = '''
    SELECT * FROM Customers
    WHERE City = 'Berlin' OR CustomerName LIKE 'G%' OR Country = 'Norway';
'''
execute_example(query)

print("Combining AND and OR")
# Select all Spanish customers that starts with either "G" or "R":
query = '''
    SELECT * FROM Customers
    WHERE Country = 'Spain' AND (CustomerName LIKE 'G%' OR CustomerName LIKE 'R%');
'''
execute_example(query)

print("Combining AND and OR without Parentheses")
# Select all customers that either:
# are from Spain and starts with either "G", or
# starts with the letter "R":
query = '''
    SELECT * FROM Customers
    WHERE Country = 'Spain' AND CustomerName LIKE 'G%' OR CustomerName LIKE 'R%';
'''
execute_example(query)

The SQL OR Operator
CustomerID | CustomerName | ContactName | Address | City | PostalCode | Country
---------------------------------------------------------------------------------------------------------
1 | Alfreds Futterkiste | Maria Anders | Obere Str. 57 | Berlin | 12209 | Germany
6 | Blauer See Delikatessen | Hanna Moos | Forsterstr. 57 | Mannheim | 68306 | Germany
8 | Bólido Comidas preparadas | Martín Sommer | C/ Araquil, 67 | Madrid | 28023 | Spain
17 | Drachenblut Delikatessend | Sven Ottlieb | Walserweg 21 | Aachen | 52066 | Germany
22 | FISSA Fabrica Inter. Salchichas S.A. | Diego Roel | C/ Moralzarzal, 86 | Madrid | 28034 | Spain
25 | Frankenversand | Peter Franken | Berliner Platz 43 | München | 80805 | Germany
29 | Galería del gastrónomo | Eduardo Saavedra | Rambla de Cataluña, 23 | Barcelona | 8022 | Spain
30 | Godos Cocina Típica | José Pedro Freyre | C/ Romero, 33 | Sevilla | 41101 | Spain
39 | Königlich Essen | Philip Cramer | Maubelstr. 90 | Brandenburg | 14776 | G

### The NOT Operator
1. The NOT operator is used in combination with other operators to give the opposite result, also called the negative result.

In [571]:
print("The SQL NOT Operator")
# Select only the customers that are NOT from Spain:
query = '''
    SELECT * FROM Customers
    WHERE NOT Country = 'Spain';
'''
execute_example(query, N=5)

print("The SQL NOT LIKE Operator")
# Select customers that does not start with the letter 'A':
query = '''
    SELECT * FROM Customers
    WHERE CustomerName NOT LIKE 'A%';
'''
execute_example(query, N=5)

print("The SQL NOT BETWEEN Operator")
# Select customers with a customerID not between 10 and 60:
query = '''
    SELECT * FROM Customers
    WHERE CustomerID NOT BETWEEN 10 AND 60;
'''
execute_example(query, N=5)

print("NOT IN")
# Select customers that are not from Paris or London:
query = '''
    SELECT * FROM Customers
    WHERE City NOT IN ('Paris', 'London');
'''
execute_example(query, N=5)

print("NOT Greater Than")
# Select customers with a CustomerId not greater than 50:
query = '''
    SELECT * FROM Customers
    WHERE NOT CustomerID > 50;
'''
execute_example(query, N=5)

print("NOT Less Than")
# Select customers with a CustomerID not less than 50:
query = '''
    SELECT * FROM Customers
WHERE NOT CustomerId < 50;
'''
execute_example(query, N=5)

The SQL NOT Operator
CustomerID | CustomerName | ContactName | Address | City | PostalCode | Country
---------------------------------------------------------------------------------------------------------
1 | Alfreds Futterkiste | Maria Anders | Obere Str. 57 | Berlin | 12209 | Germany
2 | Ana Trujillo Emparedados y helados | Ana Trujillo | Avda. de la Constitución 2222 | México D.F. | 5021 | Mexico
3 | Antonio Moreno Taquería | Antonio Moreno | Mataderos 2312 | México D.F. | 5023 | Mexico
4 | Around the Horn | Thomas Hardy | 120 Hanover Sq. | London | WA1 1DP | UK
5 | Berglunds snabbköp | Christina Berglund | Berguvsvägen 8 | Luleå | S-958 22 | Sweden

The SQL NOT LIKE Operator
CustomerID | CustomerName | ContactName | Address | City | PostalCode | Country
---------------------------------------------------------------------------------------------------------
5 | Berglunds snabbköp | Christina Berglund | Berguvsvägen 8 | Luleå | S-958 22 | Sweden
6 | Blauer See Delikatessen | Hanna

### SQL INSERT INTO Statement

In [572]:
print("INSERT INTO")
insert_query = '''
    INSERT INTO Customers (CustomerName, ContactName, Address, City, PostalCode, Country)
    VALUES ('Cardinal', 'Tom B. Erichsen', 'Skagen 21', 'Stavanger', '4006', 'Norway');
'''
select_query = '''
    SELECT * FROM Customers
    ORDER BY CustomerID DESC
    LIMIT 1;
'''
execute_rollback_example(insert_query, select_query)

print("Insert Data Only in Specified Columns")
insert_query = '''
    INSERT INTO Customers (CustomerName, City, Country)
    VALUES ('Cardinal', 'Stavanger', 'Norway');
'''
select_query = '''
    SELECT * FROM Customers
    ORDER BY CustomerID DESC
    LIMIT 1;
'''
execute_rollback_example(insert_query, select_query)

print("Insert Multiple Rows")
insert_query = '''
    INSERT INTO Customers (CustomerName, ContactName, Address, City, PostalCode, Country)
    VALUES
    ('Cardinal', 'Tom B. Erichsen', 'Skagen 21', 'Stavanger', '4006', 'Norway'),
    ('Greasy Burger', 'Per Olsen', 'Gateveien 15', 'Sandnes', '4306', 'Norway'),
    ('Tasty Tee', 'Finn Egan', 'Streetroad 19B', 'Liverpool', 'L1 0AA', 'UK');
'''
select_query = '''
    SELECT * FROM Customers
    ORDER BY CustomerID DESC
    LIMIT 3;
'''
execute_rollback_example(insert_query, select_query)

INSERT INTO
CustomerID | CustomerName | ContactName | Address | City | PostalCode | Country
---------------------------------------------------------------------------------------------------------
92 | Cardinal | Tom B. Erichsen | Skagen 21 | Stavanger | 4006 | Norway

Insert Data Only in Specified Columns
CustomerID | CustomerName | ContactName | Address | City | PostalCode | Country
---------------------------------------------------------------------------------------------------------
92 | Cardinal | None | None | Stavanger | None | Norway

Insert Multiple Rows
CustomerID | CustomerName | ContactName | Address | City | PostalCode | Country
---------------------------------------------------------------------------------------------------------
94 | Tasty Tee | Finn Egan | Streetroad 19B | Liverpool | L1 0AA | UK
93 | Greasy Burger | Per Olsen | Gateveien 15 | Sandnes | 4306 | Norway
92 | Cardinal | Tom B. Erichsen | Skagen 21 | Stavanger | 4006 | Norway



### SQL NULL Values
1. It is not possible to test for NULL values with comparison operators, such as =, <, or <>.

In [573]:
print("The IS NULL Operator")
# The IS NULL operator is used to test for empty values (NULL values).
insert_query = '''
    INSERT INTO Customers (CustomerName, City, Country)
    VALUES ('Cardinal', 'Stavanger', 'Norway');
'''
select_query = '''
    SELECT CustomerName, ContactName, Address
    FROM Customers
    WHERE Address IS NULL;
'''
execute_rollback_example(insert_query, select_query)

print("The IS NOT NULL Operator")
# The IS NOT NULL operator is used to test for non-empty values (NOT NULL values).
query = '''
    SELECT CustomerName, ContactName, Address
    FROM Customers
    WHERE Address IS NOT NULL;
'''
execute_example(query, N=5)

The IS NULL Operator
CustomerName | ContactName | Address
---------------------------------------------
Cardinal | None | None

The IS NOT NULL Operator
CustomerName | ContactName | Address
---------------------------------------------
Alfreds Futterkiste | Maria Anders | Obere Str. 57
Ana Trujillo Emparedados y helados | Ana Trujillo | Avda. de la Constitución 2222
Antonio Moreno Taquería | Antonio Moreno | Mataderos 2312
Around the Horn | Thomas Hardy | 120 Hanover Sq.
Berglunds snabbköp | Christina Berglund | Berguvsvägen 8



### The SQL UPDATE Statement
1. The UPDATE statement is used to modify the existing records in a table.

In [574]:
print("UPDATE Table")
update_query = '''
    UPDATE Customers
    SET ContactName = 'Alfred Schmidt', City= 'Frankfurt'
    WHERE CustomerID = 1;
'''
select_query = '''
    SELECT * FROM Customers
    WHERE CustomerID = 1;
'''
execute_rollback_example(update_query, select_query)

print("UPDATE Multiple Records")
update_query = '''
    UPDATE Customers
    SET ContactName='Juan'
    WHERE Country='Mexico';
'''
select_query = '''
    SELECT * FROM Customers
    LIMIT 5;
'''
execute_rollback_example(update_query, select_query)

print("Update Warning!")
update_query = '''
    UPDATE Customers
    SET ContactName='Juan';
'''
select_query = '''
    SELECT * FROM Customers
    LIMIT 5;
'''
execute_rollback_example(update_query, select_query)

UPDATE Table
CustomerID | CustomerName | ContactName | Address | City | PostalCode | Country
---------------------------------------------------------------------------------------------------------
1 | Alfreds Futterkiste | Alfred Schmidt | Obere Str. 57 | Frankfurt | 12209 | Germany

UPDATE Multiple Records
CustomerID | CustomerName | ContactName | Address | City | PostalCode | Country
---------------------------------------------------------------------------------------------------------
1 | Alfreds Futterkiste | Maria Anders | Obere Str. 57 | Berlin | 12209 | Germany
2 | Ana Trujillo Emparedados y helados | Juan | Avda. de la Constitución 2222 | México D.F. | 5021 | Mexico
3 | Antonio Moreno Taquería | Juan | Mataderos 2312 | México D.F. | 5023 | Mexico
4 | Around the Horn | Thomas Hardy | 120 Hanover Sq. | London | WA1 1DP | UK
5 | Berglunds snabbköp | Christina Berglund | Berguvsvägen 8 | Luleå | S-958 22 | Sweden

Update Warning!
CustomerID | CustomerName | ContactName | Addres

### The SQL DELETE Statement

In [575]:
print("DELETE a customer")
# Deletes the customer "Alfreds Futterkiste" from the "Customers" table
delete_query = '''
    DELETE FROM Customers WHERE CustomerName='Alfreds Futterkiste';
'''
select_query = '''
    SELECT * FROM Customers
    LIMIT 5;
'''
execute_rollback_example(delete_query, select_query)

print("Delete All Records")
delete_query = '''
    DELETE FROM Customers;
'''
select_query = '''
     SELECT * FROM Customers
'''
execute_rollback_example(delete_query, select_query)

print("Delete a Table")
drop_query = '''
    DROP TABLE Customers;
'''
# DROP TABLE은 롤백이 불가능하다.
# execute_example(drop_query)

DELETE a customer
CustomerID | CustomerName | ContactName | Address | City | PostalCode | Country
---------------------------------------------------------------------------------------------------------
2 | Ana Trujillo Emparedados y helados | Ana Trujillo | Avda. de la Constitución 2222 | México D.F. | 5021 | Mexico
3 | Antonio Moreno Taquería | Antonio Moreno | Mataderos 2312 | México D.F. | 5023 | Mexico
4 | Around the Horn | Thomas Hardy | 120 Hanover Sq. | London | WA1 1DP | UK
5 | Berglunds snabbköp | Christina Berglund | Berguvsvägen 8 | Luleå | S-958 22 | Sweden
6 | Blauer See Delikatessen | Hanna Moos | Forsterstr. 57 | Mannheim | 68306 | Germany

Delete All Records
CustomerID | CustomerName | ContactName | Address | City | PostalCode | Country
---------------------------------------------------------------------------------------------------------

Delete a Table


### SQL TOP, LIMIT, FETCH FIRST or ROWNUM Clause
Q. 각 KEYWORD 간의 내부 동작 차이가 무엇인가?

In [576]:
"""
print("The SQL SELECT TOP Clause")
# Select only the first 3 records of the Customers table:
query = '''
    SELECT TOP 3 * FROM Customers;
'''
execute_example(query) # for SQL Server/MS Access
"""

print("LIMIT")
# Select only the first 3 records of the Customers table:
query = '''
    SELECT * FROM Customers
    LIMIT 3;
'''
execute_example(query)

"""
print("FETCH FIRST")
query = '''
    SELECT * FROM Customers
    FETCH FIRST 3 ROWS ONLY;
'''
execute_example(query) # for Oracle
"""

"""
print("SQL TOP PERCENT Example")
# Select only the first 3 records of the Customers table:
query = '''
    SELECT TOP 50 PERCENT * FROM Customers;
'''
execute_example(query) # for SQL Server/MS Access
"""

print("ADD a WHERE CLAUSE")
query = '''
    SELECT * FROM Customers
    WHERE Country='Germany'
    LIMIT 3;
'''
execute_example(query)

print("ADD the ORDER BY Keyword")
query = '''
    SELECT * FROM Customers
    ORDER BY CustomerName DESC
    LIMIT 3;
'''
execute_example(query)

LIMIT
CustomerID | CustomerName | ContactName | Address | City | PostalCode | Country
---------------------------------------------------------------------------------------------------------
1 | Alfreds Futterkiste | Maria Anders | Obere Str. 57 | Berlin | 12209 | Germany
2 | Ana Trujillo Emparedados y helados | Ana Trujillo | Avda. de la Constitución 2222 | México D.F. | 5021 | Mexico
3 | Antonio Moreno Taquería | Antonio Moreno | Mataderos 2312 | México D.F. | 5023 | Mexico

ADD a WHERE CLAUSE
CustomerID | CustomerName | ContactName | Address | City | PostalCode | Country
---------------------------------------------------------------------------------------------------------
1 | Alfreds Futterkiste | Maria Anders | Obere Str. 57 | Berlin | 12209 | Germany
6 | Blauer See Delikatessen | Hanna Moos | Forsterstr. 57 | Mannheim | 68306 | Germany
17 | Drachenblut Delikatessend | Sven Ottlieb | Walserweg 21 | Aachen | 52066 | Germany

ADD the ORDER BY Keyword
CustomerID | CustomerName | C

### SQL Aggregate Functions
- An aggregate function is a function that performs a calculation on a set of values, and returns a single value.
- Aggregate functions are often used with the GROUP BY clause of the SELECT statement.
- The GROUP BY clause splits the result-set into groups of values and the aggregate function can be used to return a single value for each group.

### Commonly used SQL aggregate functions
1. MIN()
2. MAX()
3. COUNT()
4. SUM()
5. AVG()

In [577]:
print("MIN()")
query = '''
    SELECT MIN(Price)
    FROM Products;
'''
execute_example(query)

print("MAX()")
query = '''
    SELECT MAX(Price)
    FROM Products;
'''
execute_example(query)

print("Set Column Name (Alias)")
query = '''
    SELECT MIN(Price) AS SmallestPrice
    FROM Products;
'''
execute_example(query)

print("Use MIN() with GROUP BY")
query = '''
    SELECT MIN(Price) AS SmallestPrice, CategoryID
    FROM Products
    GROUP BY CategoryID;
'''
execute_example(query)

MIN()
MIN(Price)
---------------
2.5

MAX()
MAX(Price)
---------------
263.5

Set Column Name (Alias)
SmallestPrice
---------------
2.5

Use MIN() with GROUP BY
SmallestPrice | CategoryID
------------------------------
4.5 | 1
10 | 2
9.2 | 3
2.5 | 4
7 | 5
7.45 | 6
10 | 7
6 | 8



In [578]:
print("COUNT()")
query = '''
    SELECT COUNT(*)
    FROM Products;
'''
execute_example(query)

print("COUNT()")
query = '''
    SELECT COUNT(ProductName)
    FROM Products;
'''
execute_example(query)

print("Add a WHERE Clause")
query = '''
    SELECT COUNT(ProductID)
    FROM Products
    WHERE Price > 20;
'''
execute_example(query)

print("Ignore Duplicates")
query = '''
    SELECT COUNT(DISTINCT Price)
    FROM Products;
'''
execute_example(query)

print("Use an Alias")
query = '''
    SELECT COUNT(*) AS [Number of records]
    FROM Products;
'''
execute_example(query)

print("Use COUNT() with GROUP BY")
query = '''
    SELECT COUNT(*) AS [Number of records], CategoryID
    FROM Products
    GROUP BY CategoryID;
'''
execute_example(query)

COUNT()
COUNT(*)
---------------
77

COUNT()
COUNT(ProductName)
---------------
77

Add a WHERE Clause
COUNT(ProductID)
---------------
37

Ignore Duplicates
COUNT(DISTINCT Price)
---------------
62

Use an Alias
Number of records
---------------
77

Use COUNT() with GROUP BY
Number of records | CategoryID
------------------------------
12 | 1
12 | 2
13 | 3
10 | 4
7 | 5
6 | 6
5 | 7
12 | 8



In [579]:
print("SUM()")
# Return the sum of all Quantity fields in the OrderDetails table:
query = '''
    SELECT SUM(Quantity)
    FROM OrderDetails;
'''
execute_example(query)

print("Add a WHERE Clause")
# Return the sum of the Quantity field for the product with ProductID 11:
query = '''
    SELECT SUM(Quantity)
    FROM OrderDetails
    WHERE ProductId = 11;
'''
execute_example(query)

print("Use an Alias")
query = '''
    SELECT SUM(Quantity) AS total
    FROM OrderDetails;
'''
execute_example(query)

print("Use SUM() with GROUP BY")
query = '''
    SELECT OrderID, SUM(Quantity) AS [Total Quantity]
    FROM OrderDetails
    GROUP BY OrderID;
'''
execute_example(query, N=5)

print("SUM() With an Expression")
query = '''
    SELECT SUM(Quantity * 10)
    FROM OrderDetails;
'''
execute_example(query)

print("Join OrderDetails with Products, and use SUM() to find the total amount")
query = '''
    SELECT SUM(Price * Quantity)
    FROM OrderDetails
    LEFT JOIN Products ON OrderDetails.ProductID = Products.ProductID;
'''
execute_example(query)

SUM()
SUM(Quantity)
---------------
12743

Add a WHERE Clause
SUM(Quantity)
---------------
182

Use an Alias
total
---------------
12743

Use SUM() with GROUP BY
OrderID | Total Quantity
------------------------------
10248 | 27
10249 | 49
10250 | 60
10251 | 41
10252 | 105

SUM() With an Expression
SUM(Quantity * 10)
---------------
127430

Join OrderDetails with Products, and use SUM() to find the total amount
SUM(Price * Quantity)
---------------
386424.23



In [580]:
print("AVG()")
query = '''
    SELECT AVG(Price)
    FROM Products;
'''
execute_example(query)

print("Add a WHERE Clause")
# Return the sum of the Quantity field for the product with ProductID 11:
query = '''
    SELECT AVG(Price)
    FROM Products
    WHERE CategoryID = 1;
'''
execute_example(query)

print("Use an Alias")
query = '''
    SELECT AVG(Price) AS [average price]
    FROM Products;
'''
execute_example(query)

print("Higher Than Average")
query = '''
    SELECT * FROM Products
    WHERE price > (SELECT AVG(price) FROM Products);
'''
execute_example(query, N=5)

print("Use AVG() with GROUP BY")
query = '''
    SELECT AVG(Price) AS AveragePrice, CategoryID
    FROM Products
    GROUP BY CategoryID;
'''
execute_example(query)

AVG()
AVG(Price)
---------------
28.866363636363637

Add a WHERE Clause
AVG(Price)
---------------
37.979166666666664

Use an Alias
average price
---------------
28.866363636363637

Higher Than Average
ProductID | ProductName | SupplierID | CategoryID | Unit | Price
------------------------------------------------------------------------------------------
7 | Uncle Bob's Organic Dried Pears | 3 | 7 | 12 - 1 lb pkgs. | 30
8 | Northwoods Cranberry Sauce | 3 | 2 | 12 - 12 oz jars | 40
9 | Mishi Kobe Niku | 4 | 6 | 18 - 500 g pkgs. | 97
10 | Ikura | 4 | 8 | 12 - 200 ml jars | 31
12 | Queso Manchego La Pastora | 5 | 4 | 10 - 500 g pkgs. | 38

Use AVG() with GROUP BY
AveragePrice | CategoryID
------------------------------
37.979166666666664 | 1
23.0625 | 2
25.16 | 3
28.73 | 4
20.25 | 5
54.00666666666667 | 6
32.37 | 7
20.6825 | 8



### The SQL LIKE Operator
1. The percent sign % represents zero, one, or multiple characters
2. The underscore sign _ represents one, single character

In [581]:
print("The SQL LIKE Operator")
query = '''
    SELECT * FROM Customers
    WHERE CustomerName LIKE 'a%';
'''
execute_example(query, N=5)

print("The _ Wildcard")
query = '''
   SELECT * FROM Customers
    WHERE city LIKE 'L_nd__';
'''
execute_example(query, N=5)

print("The % Wildcard")
query = '''
    SELECT * FROM Customers
    WHERE city LIKE '%L%';
'''
execute_example(query, N=5)

print("Starts With")
query = '''
    SELECT * FROM Customers
    WHERE CustomerName LIKE 'La%';
'''
execute_example(query, N=5)

print("Starts With using AND or OR operators.")
query = '''
    SELECT * FROM Customers
    WHERE CustomerName LIKE 'a%' OR CustomerName LIKE 'b%';
'''
execute_example(query, N=5)

print("Ends With")
query = '''
    SELECT * FROM Customers
    WHERE CustomerName LIKE '%a';
'''
execute_example(query, N=5)

print("Starts With and Ends With")
query = '''
    SELECT * FROM Customers
    WHERE CustomerName LIKE 'b%s';
'''
execute_example(query, N=5)

print("Contains")
query = '''
    SELECT * FROM Customers
    WHERE CustomerName LIKE '%or%';
'''
execute_example(query, N=5)

print("Combine Wildcards case 1")
query = '''
    SELECT * FROM Customers
    WHERE CustomerName LIKE 'a__%';
'''
execute_example(query, N=5)

print("Combine Wildcards case 2")
query = '''
    SELECT * FROM Customers
    WHERE CustomerName LIKE '_r%';
'''
execute_example(query, N=5)

print("Without Wildcard")
query = '''
    SELECT * FROM Customers
    WHERE Country LIKE 'Spain';
'''
execute_example(query, N=5)

The SQL LIKE Operator
CustomerID | CustomerName | ContactName | Address | City | PostalCode | Country
---------------------------------------------------------------------------------------------------------
1 | Alfreds Futterkiste | Maria Anders | Obere Str. 57 | Berlin | 12209 | Germany
2 | Ana Trujillo Emparedados y helados | Ana Trujillo | Avda. de la Constitución 2222 | México D.F. | 5021 | Mexico
3 | Antonio Moreno Taquería | Antonio Moreno | Mataderos 2312 | México D.F. | 5023 | Mexico
4 | Around the Horn | Thomas Hardy | 120 Hanover Sq. | London | WA1 1DP | UK

The _ Wildcard
CustomerID | CustomerName | ContactName | Address | City | PostalCode | Country
---------------------------------------------------------------------------------------------------------
4 | Around the Horn | Thomas Hardy | 120 Hanover Sq. | London | WA1 1DP | UK
11 | B''s Beverages | Victoria Ashworth | Fauntleroy Circus | London | EC2 5NT | UK
16 | Consolidated Holdings | Elizabeth Brown | Berkeley Garden

### SQL Wildcard Characters
1. %	Represents zero or more characters
2. _	Represents a single character
3. []	Represents any single character within the brackets *
4. ^	Represents any character not in the brackets
5. \-	Represents any single character within the specified range *
6. {}	Represents any escaped character

In [582]:
print("SQL Wildcard Characters")
query = '''
    SELECT * FROM Customers
    WHERE CustomerName LIKE 'a%';
'''
execute_example(query, N=5)

print("Using the % Wildcard")
query = '''
    SELECT * FROM Customers
    WHERE CustomerName LIKE '%es';
'''
execute_example(query, N=5)

print("Using the _ Wildcard case 1")
query = '''
    SELECT * FROM Customers
    WHERE City LIKE '_ondon';
'''
execute_example(query, N=5)

print("Using the _ Wildcard case 2")
query = '''
    SELECT * FROM Customers
    WHERE City LIKE 'L___on';
'''
execute_example(query, N=5)

print("Using the - Wildcard")
query = '''
    SELECT * FROM Customers
    WHERE CustomerName LIKE '[a-f]%';
'''
execute_example(query, N=5)

print("Combine Wildcards case 1")
query = '''
    SELECT * FROM Customers
    WHERE CustomerName LIKE 'a__%';
'''
execute_example(query, N=5)

print("Combine Wildcards case 2")
query = '''
    SELECT * FROM Customers
    WHERE CustomerName LIKE '_r%';
'''
execute_example(query, N=5)

print("Without Wildcard")
query = '''
    SELECT * FROM Customers
    WHERE Country LIKE 'Spain';
'''
execute_example(query, N=5)

SQL Wildcard Characters
CustomerID | CustomerName | ContactName | Address | City | PostalCode | Country
---------------------------------------------------------------------------------------------------------
1 | Alfreds Futterkiste | Maria Anders | Obere Str. 57 | Berlin | 12209 | Germany
2 | Ana Trujillo Emparedados y helados | Ana Trujillo | Avda. de la Constitución 2222 | México D.F. | 5021 | Mexico
3 | Antonio Moreno Taquería | Antonio Moreno | Mataderos 2312 | México D.F. | 5023 | Mexico
4 | Around the Horn | Thomas Hardy | 120 Hanover Sq. | London | WA1 1DP | UK

Using the % Wildcard
CustomerID | CustomerName | ContactName | Address | City | PostalCode | Country
---------------------------------------------------------------------------------------------------------
11 | B''s Beverages | Victoria Ashworth | Fauntleroy Circus | London | EC2 5NT | UK
23 | Folies gourmandes | Martine Rancé | 184, chaussée de Tournai | Lille | 59000 | France
31 | Gourmet Lanchonetes | André Fonseca

### The SQL IN Operator

In [583]:
print("The SQL IN Operator")
query = '''
    SELECT * FROM Customers
    WHERE Country IN ('Germany', 'France', 'UK');
'''
execute_example(query, N=5)

print("NOT IN")
query = '''
    SELECT * FROM Customers
    WHERE Country NOT IN ('Germany', 'France', 'UK');
'''
execute_example(query, N=5)

print("IN (SELECT)")
query = '''
    SELECT * FROM Customers
    WHERE CustomerID IN (SELECT CustomerID FROM Orders);
'''
execute_example(query, N=5)

print("NOT IN (SELECT)")
query = '''
    SELECT * FROM Customers
    WHERE CustomerID NOT IN (SELECT CustomerID FROM Orders);
'''
execute_example(query, N=5) 

The SQL IN Operator
CustomerID | CustomerName | ContactName | Address | City | PostalCode | Country
---------------------------------------------------------------------------------------------------------
1 | Alfreds Futterkiste | Maria Anders | Obere Str. 57 | Berlin | 12209 | Germany
4 | Around the Horn | Thomas Hardy | 120 Hanover Sq. | London | WA1 1DP | UK
6 | Blauer See Delikatessen | Hanna Moos | Forsterstr. 57 | Mannheim | 68306 | Germany
7 | Blondel père et fils | Frédérique Citeaux | 24, place Kléber | Strasbourg | 67000 | France
9 | Bon app'' | Laurence Lebihans | 12, rue des Bouchers | Marseille | 13008 | France

NOT IN
CustomerID | CustomerName | ContactName | Address | City | PostalCode | Country
---------------------------------------------------------------------------------------------------------
2 | Ana Trujillo Emparedados y helados | Ana Trujillo | Avda. de la Constitución 2222 | México D.F. | 5021 | Mexico
3 | Antonio Moreno Taquería | Antonio Moreno | Mataderos 

### The SQL BETWEEN Operator
1. The BETWEEN operator selects values within a given range. The values can be numbers, text, or dates.

In [584]:
print("The SQL BETWEEN Operator")
query = '''
    SELECT * FROM Products
    WHERE Price BETWEEN 10 AND 20;
'''
execute_example(query, N=5)

print("NOT BETWEEN")
query = '''
    SELECT * FROM Products
    WHERE Price NOT BETWEEN 10 AND 20;
'''
execute_example(query, N=5)

print("BETWEEN with IN")
query = '''
    SELECT * FROM Products
    WHERE Price BETWEEN 10 AND 20
    AND CategoryID IN (1,2,3);
'''
execute_example(query, N=5)

print("BETWEEN Text Values")
query = '''
    SELECT * FROM Products
    WHERE ProductName BETWEEN 'Carnarvon Tigers' AND 'Mozzarella di Giovanni'
    ORDER BY ProductName;
'''
execute_example(query, N=5)

print("BETWEEN Text Values case 2")
query = '''
    SELECT * FROM Products
    WHERE ProductName BETWEEN "Carnarvon Tigers" AND "Chef Anton's Cajun Seasoning"
    ORDER BY ProductName;
'''
execute_example(query, N=5)

print("NOT BETWEEN Text Values")
query = '''
    SELECT * FROM Products
    WHERE ProductName NOT BETWEEN 'Carnarvon Tigers' AND 'Mozzarella di Giovanni'
    ORDER BY ProductName;
'''
execute_example(query, N=5) 

print("BETWEEN Dates case")
query = '''
    SELECT * FROM Orders
    WHERE OrderDate BETWEEN '1996-07-01' AND '1996-07-31';
'''
execute_example(query, N=5)

The SQL BETWEEN Operator
ProductID | ProductName | SupplierID | CategoryID | Unit | Price
------------------------------------------------------------------------------------------
1 | Chais | 1 | 1 | 10 boxes x 20 bags | 18
2 | Chang | 1 | 1 | 24 - 12 oz bottles | 19
3 | Aniseed Syrup | 1 | 2 | 12 - 550 ml bottles | 10
15 | Genen Shouyu | 6 | 2 | 24 - 250 ml bottles | 15.5
16 | Pavlova | 7 | 3 | 32 - 500 g boxes | 17.45

NOT BETWEEN
ProductID | ProductName | SupplierID | CategoryID | Unit | Price
------------------------------------------------------------------------------------------
4 | Chef Anton's Cajun Seasoning | 2 | 2 | 48 - 6 oz jars | 22
5 | Chef Anton's Gumbo Mix | 2 | 2 | 36 boxes | 21.35
6 | Grandma's Boysenberry Spread | 3 | 2 | 12 - 8 oz jars | 25
7 | Uncle Bob's Organic Dried Pears | 3 | 7 | 12 - 1 lb pkgs. | 30
8 | Northwoods Cranberry Sauce | 3 | 2 | 12 - 12 oz jars | 40

BETWEEN with IN
ProductID | ProductName | SupplierID | CategoryID | Unit | Price
---------------

### SQL Aliases

In [585]:
print("SQL Aliases")
query = '''
    SELECT CustomerID AS ID
    FROM Customers;
'''
execute_example(query, N=5)

print("AS is Optional")
query = '''
    SELECT CustomerID ID
    FROM Customers;
'''
execute_example(query, N=5)

print("Alias for Columns")
query = '''
    SELECT CustomerID AS ID, CustomerName AS Customer
    FROM Customers;
'''
execute_example(query, N=5)

print("Using Aliases With a Space Character")
query = '''
    SELECT ProductName AS [My Great Products]
    FROM Products;
'''
execute_example(query, N=5)

print("Using Aliases With a Space Character case 2")
query = '''
    SELECT ProductName AS "My Great Products"
    FROM Products;
'''
execute_example(query, N=5)

print("Concatenate Columns")
query = '''
    SELECT CustomerName, Address + ', ' + PostalCode + ' ' + City + ', ' + Country AS Address
    FROM Customers;
'''
execute_example(query, N=5)

print("MYSQL Concatenate Columns")
query = '''
    SELECT CustomerName, CONCAT(Address,', ',PostalCode,', ',City,', ',Country) AS Address
    FROM Customers;
'''
execute_example(query, N=5)

print("Alias for Tables")
query = '''
    SELECT * FROM Customers AS Persons;
'''
execute_example(query, N=5) 

print("Complex Alias for Tables")
query = '''
    SELECT o.OrderID, o.OrderDate, c.CustomerName
    FROM Customers AS c, Orders AS o
    WHERE c.CustomerName='Around the Horn' AND c.CustomerID=o.CustomerID;
'''
execute_example(query, N=5) 

print("Without Alias for Tables")
query = '''
    SELECT Orders.OrderID, Orders.OrderDate, Customers.CustomerName
    FROM Customers, Orders
    WHERE Customers.CustomerName='Around the Horn' AND Customers.CustomerID=Orders.CustomerID;
'''
execute_example(query, N=5)

SQL Aliases
ID
---------------
1
2
3
4
5

AS is Optional
ID
---------------
1
2
3
4
5

Alias for Columns
ID | Customer
------------------------------
1 | Alfreds Futterkiste
2 | Ana Trujillo Emparedados y helados
3 | Antonio Moreno Taquería
4 | Around the Horn
5 | Berglunds snabbköp

Using Aliases With a Space Character
My Great Products
---------------
Chais
Chang
Aniseed Syrup
Chef Anton's Cajun Seasoning
Chef Anton's Gumbo Mix

Using Aliases With a Space Character case 2
My Great Products
---------------
Chais
Chang
Aniseed Syrup
Chef Anton's Cajun Seasoning
Chef Anton's Gumbo Mix

Concatenate Columns
CustomerName | Address
------------------------------
Alfreds Futterkiste | 12209
Ana Trujillo Emparedados y helados | 5021
Antonio Moreno Taquería | 5023
Around the Horn | 120
Berglunds snabbköp | 0

MYSQL Concatenate Columns
CustomerName | Address
------------------------------
Alfreds Futterkiste | Obere Str. 57, 12209, Berlin, Germany
Ana Trujillo Emparedados y helados | Avda. de l

### SQL JOIN
- A JOIN clause is used to combine rows from two or more tables, based on a related column between them.
- (INNER) JOIN: Returns records that have matching values in both tables
- LEFT (OUTER) JOIN: Returns all records from the left table, and the matched records from the right table
- RIGHT (OUTER) JOIN: Returns all records from the right table, and the matched records from the left table
- FULL (OUTER) JOIN: Returns all records when there is a match in either left or right table

### INNER JOIN

In [586]:
print("INNTER JOIN")
query = '''
    SELECT ProductID, ProductName, CategoryName
    FROM Products
    INNER JOIN Categories ON Products.CategoryID = Categories.CategoryID;
'''
execute_example(query, N=5)

print("Naming the Columns")
query = '''
    SELECT Products.ProductID, Products.ProductName, Categories.CategoryName
    FROM Products
    INNER JOIN Categories ON Products.CategoryID = Categories.CategoryID;
'''
execute_example(query, N=5)

print("JOIN or INNER JOIN")
query = '''
    SELECT Products.ProductID, Products.ProductName, Categories.CategoryName
    FROM Products
    JOIN Categories ON Products.CategoryID = Categories.CategoryID;
'''
execute_example(query, N=5)

print("JOIN Three Tables")
query = '''
    SELECT Orders.OrderID, Customers.CustomerName, Shippers.ShipperName
    FROM ((Orders
    INNER JOIN Customers ON Orders.CustomerID = Customers.CustomerID)
    INNER JOIN Shippers ON Orders.ShipperID = Shippers.ShipperID);
'''
execute_example(query, N=5)


INNTER JOIN
ProductID | ProductName | CategoryName
---------------------------------------------
1 | Chais | Beverages
2 | Chang | Beverages
3 | Aniseed Syrup | Condiments
4 | Chef Anton's Cajun Seasoning | Condiments
5 | Chef Anton's Gumbo Mix | Condiments

Naming the Columns
ProductID | ProductName | CategoryName
---------------------------------------------
1 | Chais | Beverages
2 | Chang | Beverages
3 | Aniseed Syrup | Condiments
4 | Chef Anton's Cajun Seasoning | Condiments
5 | Chef Anton's Gumbo Mix | Condiments

JOIN or INNER JOIN
ProductID | ProductName | CategoryName
---------------------------------------------
1 | Chais | Beverages
2 | Chang | Beverages
3 | Aniseed Syrup | Condiments
4 | Chef Anton's Cajun Seasoning | Condiments
5 | Chef Anton's Gumbo Mix | Condiments

JOIN Three Tables
OrderID | CustomerName | ShipperName
---------------------------------------------
10248 | Wilman Kala | Federal Shipping
10249 | Tradição Hipermercados | Speedy Express
10250 | Hanari Carnes

### LEFT JOIN

In [587]:
print("LEFT JOIN")
query = '''
    SELECT Customers.CustomerName, Orders.OrderID
    FROM Customers
    LEFT JOIN Orders ON Customers.CustomerID = Orders.CustomerID
    ORDER BY Customers.CustomerName;
'''
execute_example(query, N=5)


LEFT JOIN
CustomerName | OrderID
------------------------------
Alfreds Futterkiste | None
Ana Trujillo Emparedados y helados | 10308
Antonio Moreno Taquería | 10365
Around the Horn | 10355
Around the Horn | 10383



### RIGHT JOIN

In [588]:
print("RIGHT JOIN")
query = '''
    SELECT Orders.OrderID, Employees.LastName, Employees.FirstName
    FROM Orders
    RIGHT JOIN Employees ON Orders.EmployeeID = Employees.EmployeeID
    ORDER BY Orders.OrderID;
'''
execute_example(query, N=5)


RIGHT JOIN
OrderID | LastName | FirstName
---------------------------------------------
None | West | Adam
10248 | Buchanan | Steven
10249 | Suyama | Michael
10250 | Peacock | Margaret
10251 | Leverling | Janet



### FULL OUTER JOIN

In [589]:
print("FULL OUTER JOIN")
query = '''
    SELECT Customers.CustomerName, Orders.OrderID
    FROM Customers
    FULL OUTER JOIN Orders ON Customers.CustomerID=Orders.CustomerID
    ORDER BY Customers.CustomerName;
'''
execute_example(query, N=5)


FULL OUTER JOIN
CustomerName | OrderID
------------------------------
Alfreds Futterkiste | None
Ana Trujillo Emparedados y helados | 10308
Antonio Moreno Taquería | 10365
Around the Horn | 10355
Around the Horn | 10383



### SQL Self Join

In [590]:
print("Self JOIN")
query = '''
    SELECT A.CustomerName AS CustomerName1, B.CustomerName AS CustomerName2, A.City
    FROM Customers A, Customers B
    WHERE A.CustomerID <> B.CustomerID
    AND A.City = B.City
    ORDER BY A.City;
'''
execute_example(query, N=5)


Self JOIN
CustomerName1 | CustomerName2 | City
---------------------------------------------
Cactus Comidas para llevar | Océano Atlántico Ltda. | Buenos Aires
Cactus Comidas para llevar | Rancho grande | Buenos Aires
Océano Atlántico Ltda. | Cactus Comidas para llevar | Buenos Aires
Océano Atlántico Ltda. | Rancho grande | Buenos Aires
Rancho grande | Cactus Comidas para llevar | Buenos Aires



### SQL UNION Operator

In [591]:
print("SQL UNION")
query = '''
    SELECT City FROM Customers
    UNION
    SELECT City FROM Suppliers
    ORDER BY City;
'''
execute_example(query, N=5)

print("SQL UNION ALL")
query = '''
    SELECT City FROM Customers
    UNION ALL
    SELECT City FROM Suppliers
    ORDER BY City;
'''
execute_example(query, N=5)

print("SQL UNION With WHERE")
query = '''
    SELECT City, Country FROM Customers
    WHERE Country='Germany'
    UNION
    SELECT City, Country FROM Suppliers
    WHERE Country='Germany'
    ORDER BY City;
'''
execute_example(query, N=5)

print("SQL UNION ALL With WHERE")
query = '''
    SELECT City, Country FROM Customers
    WHERE Country='Germany'
    UNION ALL
    SELECT City, Country FROM Suppliers
    WHERE Country='Germany'
    ORDER BY City;
'''
execute_example(query, N=5)

print("Another UNION Example")
query = '''
    SELECT 'Customer' AS Type, ContactName, City, Country
    FROM Customers
    UNION
    SELECT 'Supplier', ContactName, City, Country
    FROM Suppliers;
'''
execute_example(query, N=5)


SQL UNION
City
---------------
Aachen
Albuquerque
Anchorage
Ann Arbor
Annecy

SQL UNION ALL
City
---------------
Aachen
Albuquerque
Anchorage
Ann Arbor
Annecy

SQL UNION With WHERE
City | Country
------------------------------
Aachen | Germany
Berlin | Germany
Brandenburg | Germany
Cunewalde | Germany
Cuxhaven | Germany

SQL UNION ALL With WHERE
City | Country
------------------------------
Aachen | Germany
Berlin | Germany
Berlin | Germany
Brandenburg | Germany
Cunewalde | Germany

Another UNION Example
Type | ContactName | City | Country
------------------------------------------------------------
Customer | Alejandra Camino | Madrid | Spain
Customer | Alexander Feuer | Leipzig | Germany
Customer | Ana Trujillo | México D.F. | Mexico
Customer | Anabela Domingues | São Paulo | Brazil
Customer | André Fonseca | Campinas | Brazil



### SQL GROUP BY Statement
- The GROUP BY statement is often used with aggregate functions (COUNT(), MAX(), MIN(), SUM(), AVG()) to group the result-set by one or more columns.

In [592]:
print("SQL GROUP BY")
query = '''
    SELECT COUNT(CustomerID), Country
    FROM Customers
    GROUP BY Country;
'''
execute_example(query, N=5)

print("SQL SORTED GROUP BY")
query = '''
    SELECT COUNT(CustomerID), Country
    FROM Customers
    GROUP BY Country
    ORDER BY COUNT(CustomerID) DESC;
'''
execute_example(query, N=5)

print("GROUP BY With JOIN Example")
query = '''
    SELECT Shippers.ShipperName, COUNT(Orders.OrderID) AS NumberOfOrders FROM Orders
    LEFT JOIN Shippers ON Orders.ShipperID = Shippers.ShipperID
    GROUP BY ShipperName;
'''
execute_example(query)


SQL GROUP BY
COUNT(CustomerID) | Country
------------------------------
3 | Argentina
2 | Austria
2 | Belgium
9 | Brazil
3 | Canada

SQL SORTED GROUP BY
COUNT(CustomerID) | Country
------------------------------
13 | USA
11 | Germany
11 | France
9 | Brazil
7 | UK

GROUP BY With JOIN Example
ShipperName | NumberOfOrders
------------------------------
Federal Shipping | 68
Speedy Express | 54
United Package | 74



### The SQL HAVING Clause

In [593]:
print("SQL HAVING")
query = '''
    SELECT COUNT(CustomerID), Country
    FROM Customers
    GROUP BY Country
    HAVING COUNT(CustomerID) > 5;
'''
execute_example(query, N=5)

print("SQL SORTED HAVING")
query = '''
    SELECT COUNT(CustomerID), Country
    FROM Customers
    GROUP BY Country
    HAVING COUNT(CustomerID) > 5
    ORDER BY COUNT(CustomerID) DESC;
'''
execute_example(query, N=5)

print("The employees that have registered more than 10 orders")
query = '''
   SELECT Employees.LastName, COUNT(Orders.OrderID) AS NumberOfOrders
    FROM (Orders
    INNER JOIN Employees ON Orders.EmployeeID = Employees.EmployeeID)
    GROUP BY LastName
    HAVING COUNT(Orders.OrderID) > 10;
'''
execute_example(query, N=5)

print("The employees \"Davolio\" or \"Fuller\" have registered more than 25 orders")
query = '''
    SELECT Employees.LastName, COUNT(Orders.OrderID) AS NumberOfOrders
    FROM Orders
    INNER JOIN Employees ON Orders.EmployeeID = Employees.EmployeeID
    WHERE LastName = 'Davolio' OR LastName = 'Fuller'
    GROUP BY LastName
    HAVING COUNT(Orders.OrderID) > 25;
'''
execute_example(query)


SQL HAVING
COUNT(CustomerID) | Country
------------------------------
9 | Brazil
11 | France
11 | Germany
7 | UK
13 | USA

SQL SORTED HAVING
COUNT(CustomerID) | Country
------------------------------
13 | USA
11 | Germany
11 | France
9 | Brazil
7 | UK

The employees that have registered more than 10 orders
LastName | NumberOfOrders
------------------------------
Buchanan | 11
Callahan | 27
Davolio | 29
Fuller | 20
King | 14

The employees "Davolio" or "Fuller" have registered more than 25 orders
LastName | NumberOfOrders
------------------------------
Davolio | 29



### The SQL EXISTS Operator

In [594]:
print("SQL EXISTS")
query = '''
    SELECT SupplierName
    FROM Suppliers
    WHERE EXISTS (SELECT ProductName
                    FROM Products
                    WHERE Products.SupplierID = Suppliers.supplierID AND Price < 20);
'''
execute_example(query, N=5)

print("SQL EXISTS case 2")
query = '''
    SELECT SupplierName
    FROM Suppliers
    WHERE EXISTS (SELECT ProductName
                    FROM Products
                    WHERE Products.SupplierID = Suppliers.supplierID AND Price = 22);
'''
execute_example(query)

SQL EXISTS
SupplierName
---------------
Exotic Liquid
New Orleans Cajun Delights
Tokyo Traders
Mayumi's
Pavlova, Ltd.

SQL EXISTS case 2
SupplierName
---------------
New Orleans Cajun Delights



### The SQL ANY and ALL Operators

In [595]:
"""
# SQLite does not support the SQL ANY operator.
print("SQL ANY")
query = '''
    SELECT ProductName
    FROM Products
    WHERE ProductID = ANY
        (SELECT ProductID
        FROM OrderDetails
        WHERE Quantity = 10);
'''
execute_example(query)
"""

"""
# SQLite does not support the SQL ANY operator in the WHERE clause.
print("SQL ALL in the where clause")
query = '''
    SELECT ProductName
    FROM Products
    WHERE ProductID ALL
        (SELECT ProductID
        FROM OrderDetails
        WHERE Quantity = 10);
'''
execute_example(query)
"""

print("SQL ALL")
query = '''
    SELECT ALL ProductName
    FROM Products
    WHERE TRUE;
'''
execute_example(query, N=5)


SQL ALL
ProductName
---------------
Chais
Chang
Aniseed Syrup
Chef Anton's Cajun Seasoning
Chef Anton's Gumbo Mix



### The SQL SELECT INTO Statement and INSERT INTO SELECT
- SQLite does not support SELECT INTO.

In [596]:
print("SQL SELECT INTO")
query = '''
    SELECT * INTO CustomersBackup2017
    FROM Customers;
'''

# instead of SELECT INTO
query = '''
    CREATE TEMPORARY TABLE CustomersBackup2017 AS SELECT * FROM Customers LIMIT 5;
'''
execute_rollback_example(query, "SELECT * FROM CustomersBackup2017")

# instead of SELECT INTO
execute_example("SELECT Count(*) as 'Before' FROM Customers")
insert_query = '''
    INSERT INTO Customers (CustomerName, City, Country)
    SELECT SupplierName, City, Country FROM Suppliers;
'''
execute_rollback_example(insert_query, "SELECT Count(*) as After FROM Customers")

SQL SELECT INTO
CustomerID | CustomerName | ContactName | Address | City | PostalCode | Country
---------------------------------------------------------------------------------------------------------
1 | Alfreds Futterkiste | Maria Anders | Obere Str. 57 | Berlin | 12209 | Germany
2 | Ana Trujillo Emparedados y helados | Ana Trujillo | Avda. de la Constitución 2222 | México D.F. | 5021 | Mexico
3 | Antonio Moreno Taquería | Antonio Moreno | Mataderos 2312 | México D.F. | 5023 | Mexico
4 | Around the Horn | Thomas Hardy | 120 Hanover Sq. | London | WA1 1DP | UK
5 | Berglunds snabbköp | Christina Berglund | Berguvsvägen 8 | Luleå | S-958 22 | Sweden

Before
---------------
91

After
---------------
120



### SQL CASE

In [597]:
print("SQL CASE")
query = '''
    SELECT OrderID, Quantity,
    CASE
        WHEN Quantity > 30 THEN 'The quantity is greater than 30'
        WHEN Quantity = 30 THEN 'The quantity is 30'
        ELSE 'The quantity is under 30'
    END AS QuantityText
    FROM OrderDetails;
'''
execute_example(query, N=10)

print("SQL CASE in ORDER BY")
query = '''
    SELECT CustomerName, City, Country
    FROM Customers
    ORDER BY
    (CASE
        WHEN City IS NULL THEN Country
        ELSE City
    END);
'''
execute_example(query, N=10)

SQL CASE
OrderID | Quantity | QuantityText
---------------------------------------------
10248 | 12 | The quantity is under 30
10248 | 10 | The quantity is under 30
10248 | 5 | The quantity is under 30
10249 | 9 | The quantity is under 30
10249 | 40 | The quantity is greater than 30
10250 | 10 | The quantity is under 30
10250 | 35 | The quantity is greater than 30
10250 | 15 | The quantity is under 30
10251 | 6 | The quantity is under 30
10251 | 15 | The quantity is under 30

SQL CASE in ORDER BY
CustomerName | City | Country
---------------------------------------------
Drachenblut Delikatessend | Aachen | Germany
Rattlesnake Canyon Grocery | Albuquerque | USA
Old World Delicatessen | Anchorage | USA
Galería del gastrónomo | Barcelona | Spain
LILA-Supermercado | Barquisimeto | Venezuela
Magazzini Alimentari Riuniti | Bergamo | Italy
Alfreds Futterkiste | Berlin | Germany
Chop-suey Chinese | Bern | Switzerland
Save-a-lot Markets | Boise | USA
Königlich Essen | Brandenburg | Germany



### SQL NULL Functions
- DBMS마다 지원하는 문법이 다르다.
- sqlite는 IFNULL()을 지원한다.

In [598]:
create_query = """
    CREATE TABLE Products (
        P_Id INTEGER PRIMARY KEY,
        ProductName TEXT NOT NULL,
        UnitPrice REAL NOT NULL,
        UnitsInStock INTEGER NOT NULL,
        UnitsOnOrder INTEGER
    );
"""

insert_query = """
INSERT INTO Products (P_Id, ProductName, UnitPrice, UnitsInStock, UnitsOnOrder)
VALUES(?, ?, ?, ?, ?);
"""

data = [
    (1, 'Jarlsberg', 10.45, 16, 15),
    (2, 'Mascarpone', 32.56, 23, None),
    (3, 'Gorgonzola', 15.67, 9, 20),
]

query = '''
    SELECT ProductName, UnitPrice * (UnitsInStock + IFNULL(UnitsOnOrder, 0))
    FROM Products;
'''

print("IFNULL()")
with sqlite3.connect(":memory:") as conn:
    create_demo_table(conn, create_query, insert_query, data)

    cursor = conn.cursor()
    cursor.execute(query)
    column_names = [description[0] for description in cursor.description]
    rows = cursor.fetchall()

    print_query_result(column_names, rows)
    print()

IFNULL()
ProductName | UnitPrice * (UnitsInStock + IFNULL(UnitsOnOrder, 0))
------------------------------
Jarlsberg | 323.95
Mascarpone | 748.8800000000001
Gorgonzola | 454.43



### SQL Stored Procedures

### SQL Comments
1. Single line comments start with --.
2. Multi-line comments start with /* and end with */.

### SQL Operators
1. SQL Arithmetic Operators
2. SQL Bitwise Operators
3. SQL Comparison Operators
4. SQL Compound Operators
5. SQL Logical Operators